<a href="https://colab.research.google.com/github/HwayoungYoon/Data-Analysis-Practice/blob/master/PreOnboarding-Data-Challenge/BigQuery_bquxjob_46b2397c_18cce321bd9%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'glassy-iridium-410107' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# 0. 데이터 업로드 및 확인

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=glassy-iridium-410107:US:bquxjob_46b2397c_18cce321bd9)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_46b2397c_18cce321bd9') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT *
FROM prechallenge_data.log_data
limit 10


## Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_46b2397c_18cce321bd9') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,session_id,event_name,event_property,timestamp
0,100072513,jd_view,{'page_path': '/jd'},2023-11-07 16:28:20+00:00
1,100110992,jd_view,{'page_path': '/jd'},2023-12-31 02:10:46+00:00
2,100404100,jd_view,{'page_path': '/jd'},2023-12-19 07:02:00+00:00
3,100466600,jd_view,{'page_path': '/jd'},2023-11-03 20:44:14+00:00
4,100573787,jd_view,{'page_path': '/jd'},2023-12-24 02:42:24+00:00
5,100640662,jd_view,{'page_path': '/jd'},2023-12-21 13:10:59+00:00
6,100672703,jd_view,{'page_path': '/jd'},2023-11-16 08:11:36+00:00
7,100694185,jd_view,{'page_path': '/jd'},2023-12-22 00:59:48+00:00
8,100899831,jd_view,{'page_path': '/jd'},2023-11-29 15:33:15+00:00
9,100946338,jd_view,{'page_path': '/jd'},2023-12-07 13:57:49+00:00


### Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
results.describe()

,session_id
count,10.0
mean,100548171.1
std,293073.19513
min,100072513.0
25%,100419725.0
50%,100607224.5
75%,100688814.5
max,100946338.0


# 1. 사전과제
- 1번 문제
    1. (SQL) 주(week)별 event_name=‘apply’인 session_id 수를 구하시오. week의 날짜 형식은 YYYY-MM-DD입니다. 주 시작 첫번째 날짜 기준으로 보이게 해주세요. (한 주의 시작은 일요일입니다)
    *예를 들어, 2023-12-31~2023-01-06 사이에 있는 데이터는 week = 2023-12-31에 속합니다.*
    2. (SQL) 주별 ‘_view’로 끝나는 event_name 수와 비중(%)을 구하시오. 단, ‘jd_view’는 본 분석에서 제외합니다.
    3. (SQL) 주별 (event_name=‘jd_view’인 session_id) /(event_name=[1-2. 조건]인 session_id)를 구하시오.
    4. (SQL) 주별 (event_name=‘apply’인 session_id) / (event_name=‘jd_view’인 session_id)를 구하시오.
    5. (SQL) 1-3. 1-4에서 구한 값을 week column으로 조인하시오.
- 2번 문제
    1. (SQL) event_name=‘jdlist_view’이면서 event_property.use_skill_filter=’used’인 session_id를 구하시오.
    *Hint. 구글에 ‘bigquery standard sql json_extract_scalar’를 검색해보세요*
    2. (SQL) (2-1. 조건을 만족하면서 event_name=‘jd_view’인 session_id) / (2-1. 조건을 만족하는 session_id)를 구하시오.

In [ ]:
# 1-1 문제
# 주(week)별 event_name=‘apply’인 session_id 수를 구하시오. week의 날짜 형식은 YYYY-MM-DD입니다. 주 시작 첫번째 날짜 기준으로 보이게 해주세요. (한 주의 시작은 일요일입니다)
# 예를 들어, 2023-12-31~2023-01-06 사이에 있는 데이터는 week = 2023-12-31에 속합니다.
#SELECT GENERATE_DATE_ARRAY('start', 'end', INTERVAL 1 WEEK) AS week, COUNT(session_id) AS session_num
#EXTRACT(WEEK FROM timestamp AT TIME ZONE "Asia/Seoul") AS week, COUNT(session_id) AS session_num
#FROM prechallenge_data.log_data
#WHERE event_name = 'apply'
#GROUP BY EXTRACT(WEEK FROM timestamp AT TIME ZONE "Asia/Seoul")
#

declare from_date, to_date date;

set from_date = date_sub('2023-11-01', interval 375 day);
set to_date = '2024-01-02';

SELECT date_trunc(date(base_dt), week(sunday)) as current_dd
  , format_date('%Y', base_dt) as week_year
  , format_date('%U', base_dt) as week_num, COUNT(session_id) AS session_num
FROM prechallenge_data.log_data, unnest(GENERATE_DATE_ARRAY(from_date, to_date, INTERVAL 1 WEEK)) base_dt
ORDER BY 1 desc

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=glassy-iridium-410107:US:bquxjob_46b2397c_18cce321bd9)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_46b2397c_18cce321bd9') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT *
FROM prechallenge_data.log_data
limit 10


## Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_46b2397c_18cce321bd9') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,session_id,event_name,event_property,timestamp
0,100072513,jd_view,{'page_path': '/jd'},2023-11-07 16:28:20+00:00
1,100110992,jd_view,{'page_path': '/jd'},2023-12-31 02:10:46+00:00
2,100404100,jd_view,{'page_path': '/jd'},2023-12-19 07:02:00+00:00
3,100466600,jd_view,{'page_path': '/jd'},2023-11-03 20:44:14+00:00
4,100573787,jd_view,{'page_path': '/jd'},2023-12-24 02:42:24+00:00
5,100640662,jd_view,{'page_path': '/jd'},2023-12-21 13:10:59+00:00
6,100672703,jd_view,{'page_path': '/jd'},2023-11-16 08:11:36+00:00
7,100694185,jd_view,{'page_path': '/jd'},2023-12-22 00:59:48+00:00
8,100899831,jd_view,{'page_path': '/jd'},2023-11-29 15:33:15+00:00
9,100946338,jd_view,{'page_path': '/jd'},2023-12-07 13:57:49+00:00


### Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
results.describe()

,session_id
count,10.0
mean,100548171.1
std,293073.19513
min,100072513.0
25%,100419725.0
50%,100607224.5
75%,100688814.5
max,100946338.0
